# 📘 word_full_pipeline_v7 — Word → CSV → YAML → RST → Sphinx(HTML/PDF)

**目标**：在 v6.2 的基础上，强化 HTML 呈现效果，使之更接近手册风格：
- 改进 RST 模板（admonition 信息块、code-block 示例、参数表 class）。
- 引入现代主题（`sphinx_book_theme`），并附加自定义 CSS（`_static/custom.css`）。
- 保留嵌套表优先、文本回退（生成 `valmap`）。
- 支持一键 `run_all(clean=True)`。

> 使用前将 `AT_Commands.docx` 放在与本 Notebook 同目录。

## Step 0 — 安装依赖（首次运行需要）

In [1]:
!pip install -q python-docx pandas pyyaml jinja2 sphinx sphinx_rtd_theme sphinx-book-theme lxml
print("✅ 依赖安装完成")

✅ 依赖安装完成


## Step 0.5 — 配置与通用工具（路径、日志、目录查看）

In [2]:
import os, re, json, traceback, datetime, subprocess, sys, shutil
import pandas as pd
from lxml import etree
from docx import Document
from docx.oxml.text.paragraph import CT_P
from docx.oxml.table import CT_Tbl

IN_WORD = "AT_Commands.docx"
DATA_DIR = "data"
CSV_OUT  = os.path.join(DATA_DIR, "at_extracted_commands.csv")
YAML_OUT = os.path.join(DATA_DIR, "at_all_commands.yaml")
RST_DIR  = os.path.join(DATA_DIR, "rst_output")
DOCS_DIR = "docs"
LOG_PATH = "parse_log.txt"

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RST_DIR, exist_ok=True)

def log(msg: str):
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(f"[{datetime.datetime.now().isoformat(timespec='seconds')}] {msg}\n")

open(LOG_PATH, "w", encoding="utf-8").write("")

def print_tree(root="docs"):
    if not os.path.exists(root):
        print(f"(不存在) {root}")
        return
    for dirpath, dirnames, filenames in os.walk(root):
        level = dirpath.replace(root, "").count(os.sep)
        indent = "  " * level
        print(f"{indent}{os.path.basename(dirpath)}/")
        subindent = "  " * (level + 1)
        for f in filenames:
            print(f"{subindent}{f}")
print("✅ 配置就绪；可用 print_tree('docs') 查看 Sphinx 目录结构")

✅ 配置就绪；可用 print_tree('docs') 查看 Sphinx 目录结构


## Step 1 — Word → CSV（嵌套表优先 + 文本回退）
- 识别命令标题（`AT+XXX`）；合并后续说明段；
- 识别“参数”小节，连续表格自动合并；
- 第三列优先解析嵌套表，回退文本枚举为 `valmap`。

In [3]:
CMD_LINE_PAT = re.compile(r'^\s*(AT[\+\w\-]+(?:\?[=\w<>,\s\-\+\.\:]*?)?)\s*(?::|：)?\s*(.*)$', re.I)
PARAM_HEADING_PAT = re.compile(r'^\s*参数(说明|表|信息)?\s*[:：]?\s*$', re.I)

def is_cmd_heading(text: str) -> bool: return bool(CMD_LINE_PAT.match(text or ""))
def is_param_heading(text: str) -> bool: return bool(PARAM_HEADING_PAT.match(text or ""))

def iter_ordered_blocks(doc):
    body = doc._element.body
    tbl_idx = 0
    for child in body.iterchildren():
        if isinstance(child, CT_P):
            text = "".join([t.text for t in child.xpath('.//w:t') if t.text]).strip()
            yield ("p", text)
        elif isinstance(child, CT_Tbl):
            table_obj = doc.tables[tbl_idx]
            tbl_idx += 1
            yield ("tbl", table_obj)

def cell_plain_text(cell):
    parts = [p.text.strip() for p in cell.paragraphs if p.text and p.text.strip()]
    return "\n".join(parts).strip()

def find_nested_tbls_in_cell(cell):
    xml_str = cell._tc.xml
    root = etree.fromstring(xml_str.encode("utf-8"))
    ns = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}
    return root.findall(".//w:tbl", ns), ns

def tbl_rows_as_text(tbl, ns):
    rows = []
    for r in tbl.findall(".//w:tr", ns):
        cells = r.findall(".//w:tc", ns)
        row = ["".join(tn.text for tn in c.iterfind(".//w:t", ns) if tn.text).strip() for c in cells]
        rows.append(row)
    return rows

def looks_like_header(row):
    hdr = " ".join(row[:3])
    return any(k in hdr for k in ["参数","名称","Name","描述","说明","含义","取值","值","value","meaning","备注","范围"])

def nested_table_to_valmap(rows):
    if not rows: return {}
    start = 1 if looks_like_header(rows[0]) else 0
    kv = {}
    for r in rows[start:]:
        if not r: continue
        key = (r[0] or "").strip()
        val = " | ".join([c for c in r[1:] if c and c.strip()]) if len(r) > 1 else ""
        if key: kv[key] = val
    return kv

def cell_valmap_from_nested_table(cell):
    tbls, ns = find_nested_tbls_in_cell(cell); mapping = {}
    for t in tbls:
        rows = tbl_rows_as_text(t, ns); mapping.update(nested_table_to_valmap(rows))
    return mapping

def parse_enum_map_fuzzy(text):
    if not text: return {}
    segs = re.split(r"[,\uFF0C;\uFF1B\n]+", text.strip())
    m = {}
    for s in segs:
        s = s.strip()
        if not s: continue
        if ":" in s or "：" in s:
            k, v = re.split(r"[:：]", s, 1); k, v = k.strip(), v.strip()
        else:
            m2 = re.match(r"^(\S+)\s*(?:->|→|=>|-|—|\s)\s*(.+)$", s)
            if m2: k, v = m2.group(1).strip(), m2.group(2).strip()
            else:
                m3 = re.match(r"^([A-Za-z0-9\+\-\.]+)\s+(.+)$", s)
                if m3: k, v = m3.group(1).strip(), m3.group(2).strip()
                else: continue
        if k: m[k] = v
    return m

def extract_word_to_csv(docx_path, csv_out):
    if not os.path.exists(docx_path):
        raise FileNotFoundError(f"未找到 Word 文件: {docx_path}")
    log(f"Start parsing: {docx_path}")
    doc = Document(docx_path)
    seq = list(iter_ordered_blocks(doc))

    results = []; i = 0; cmd_order = 0
    while i < len(seq):
        typ, obj = seq[i]
        if typ == "p":
            m = CMD_LINE_PAT.match(obj)
            if m:
                cmd_order += 1
                current_cmd = m.group(1).strip()
                current_title = (m.group(2) or "").strip()
                log(f"CMD[{cmd_order}] {current_cmd} — {current_title}")

                desc_lines = []; j = i + 1
                while j < len(seq):
                    t2, o2 = seq[j]
                    if t2 == "p":
                        if is_cmd_heading(o2) or is_param_heading(o2): break
                        if o2: desc_lines.append(o2)
                    elif t2 == "tbl": break
                    j += 1
                merged_desc = "\n".join(desc_lines).strip()

                params_all = []; table_count = 0; k = j
                while k < len(seq):
                    t3, o3 = seq[k]
                    if t3 == "p" and is_cmd_heading(o3): break
                    if t3 == "p" and is_param_heading(o3):
                        k += 1
                        while k < len(seq) and seq[k][0] == "tbl":
                            table = seq[k][1]; table_count += 1
                            for r in table.rows:
                                cols = r.cells
                                if not any(c.text.strip() for c in cols): continue
                                try:
                                    name = cell_plain_text(cols[0]) if len(cols) > 0 else ""
                                    desc = cell_plain_text(cols[1]) if len(cols) > 1 else ""
                                    valmap = {}
                                    if len(cols) > 2:
                                        valmap = cell_valmap_from_nested_table(cols[2]) or parse_enum_map_fuzzy(cell_plain_text(cols[2]))
                                    if not valmap and len(cols) > 1:
                                        valmap = cell_valmap_from_nested_table(cols[1]) or parse_enum_map_fuzzy(desc)
                                    if name in ("参数","参数名","Name") and any(x in desc for x in ["描述","说明","Description","Meaning"]): 
                                        continue
                                    params_all.append({"name": name, "desc": desc, "valmap": valmap})
                                except Exception as e:
                                    log(f"ROW-ERROR in {current_cmd}: {e}")
                                    log(traceback.format_exc())
                            k += 1
                        continue
                    k += 1

                if params_all or merged_desc:
                    results.append({
                        "命令": current_cmd, "命令标题": current_title, "命令类型": "执行;查询",
                        "命令格式": current_cmd, "示例命令": current_cmd, "示例响应": "",
                        "功能描述": merged_desc or current_title, "备注": "",
                        "表数量": table_count, "顺序": cmd_order,
                        "参数JSON": json.dumps(params_all, ensure_ascii=False)
                    })
                    log(f"CMD[{cmd_order}] tables={table_count} params={len(params_all)}")
                i = k; continue
        i += 1

    df = pd.DataFrame(results)
    df.to_csv(csv_out, index=False, encoding="utf-8-sig")
    print(f"✅ 提取 {len(df)} 条命令 → {csv_out}")
    print(f"📝 解析日志：{LOG_PATH}")
    return df

df_csv = extract_word_to_csv(IN_WORD, CSV_OUT)
df_csv.head()

✅ 提取 44 条命令 → data\at_extracted_commands.csv
📝 解析日志：parse_log.txt


C:\Users\txiab\AppData\Local\Temp\ipykernel_16076\3226044128.py:66: DeprecationWarning: 'maxsplit' is passed as positional argument
  k, v = re.split(r"[:：]", s, 1); k, v = k.strip(), v.strip()


,命令,命令标题,命令类型,命令格式,示例命令,示例响应,功能描述,备注,表数量,顺序,参数JSON
0,ATI,获取模组厂商信息,执行;查询,ATI,ATI,,获取模组厂商信息，包括厂家、型号和版本。\n命令格式,,1,1,"[{""name"": ""<manufacturer>"", ""desc"": ""模组厂商信息、产品..."
1,AT+GMR,查询版本信息,执行;查询,AT+GMR,AT+GMR,,查询软件版本信息。\n命令格式,,1,2,"[{""name"": ""<reversion>"", ""desc"": ""模组软件版本信息"", ""..."
2,AT+CSQ,获取信号强度,执行;查询,AT+CSQ,AT+CSQ,,查询接收信号强度<rssi>。\n命令格式,,1,3,"[{""name"": ""<signal>"", ""desc"": ""以下为signal(CSQ)与..."
3,AT+CREG,查询网络注册状态,执行;查询,AT+CREG,AT+CREG,,查询模组的当前网络注册状态。\n命令格式,,1,4,"[{""name"": ""<n>"", ""desc"": ""0：禁止网络注册主动提供结果代码（默认设..."
4,AT+CEREG,获取EPS网络注册状态,执行;查询,AT+CEREG,AT+CEREG,,查询EPS网络注册状态。\n命令格式,,1,5,"[{""name"": ""<n>"", ""desc"": ""0：禁止网络注册主动提供结果代码（默认设..."


## Step 2 — CSV → YAML（保留 valmap，增加 meta）

In [4]:
import yaml
def csv_to_yaml(csv_path, yaml_path):
    df = pd.read_csv(csv_path, dtype=str).fillna("")
    objs = []
    for _, r in df.iterrows():
        params = json.loads(r["参数JSON"]) if r["参数JSON"] else []
        objs.append({
            "command": r["命令"],
            "title": r["命令标题"],
            "type": [t.strip() for t in r["命令类型"].split(";") if t.strip()],
            "formats": [f.strip() for f in r["命令格式"].split("|") if f.strip()] or [r["命令格式"]],
            "parameters": params,
            "examples": [],
            "description": r.get("功能描述",""),
            "notes": r.get("备注",""),
            "meta": {"order": int(r.get("顺序","0") or 0), "tables": int(r.get("表数量","0") or 0)}
        })
    objs.sort(key=lambda x: x["meta"]["order"])
    with open(yaml_path, "w", encoding="utf-8") as f:
        yaml.safe_dump({"commands": objs}, f, allow_unicode=True, sort_keys=False)
    print(f"✅ 已生成 YAML → {yaml_path}")
csv_to_yaml(CSV_OUT, YAML_OUT)

✅ 已生成 YAML → data\at_all_commands.yaml


## Step 3 — YAML → RST（增强模板：admonition + code-block + CSS class）
- 顶部 `admonition` 信息块包含标题、类型、格式；  
- `Parameters` 表增加 `:class: cmd-param-table` 以便自定义 CSS；  
- `Examples` 使用 `code-block:: bash`；  
- 继续渲染 `valmap` 为嵌套表。

In [5]:
from jinja2 import Template
from collections import defaultdict
import re, os, yaml

PAGE_TMPL = Template('''
{{ cmd.command }}
{{ '=' * cmd.command|length }}

.. admonition:: {{ cmd.title }}
   :class: tip

   **类型**: {{ cmd.type|join(', ') }}
   **格式**: {{ cmd.formats|join(' | ') }}

Parameters
----------
.. list-table::
   :header-rows: 1
   :widths: 20 40 40
   :class: cmd-param-table

   * - 参数名
     - 描述
     - 取值
{%- for p in cmd.parameters %}
   * - {{ p.name }}
     - {{ p.desc or '—' }}
     - {%- if p.valmap %}
       .. list-table::
          :header-rows: 1
          :widths: 25 75

          * - 值
            - 含义
{%- for k, v in p.valmap.items() %}
          * - {{ k }}
            - {{ v }}
{%- endfor %}
       {%- else %} N/A {%- endif %}
{%- endfor %}

Examples
--------
.. code-block:: bash

{%- if cmd.examples and cmd.examples|length > 0 -%}
{%-   for ex in cmd.examples %}
   {{ ex.cmd }}
   {{ ex.resp }}
{%-   endfor %}
{%- else %}
   {{ cmd.command }}
{%- endif %}

Description
-----------
{{ cmd.description or '' }}
''')

def group_key(cmd_str):
    m = re.match(r'^AT\+([A-Z]+)', (cmd_str or "").upper())
    if not m: return "AT-OTHER"
    token = m.group(1)
    return f"AT-{token[:2]}" if len(token) >= 2 else "AT-OTHER"

def yaml_to_rst(yaml_path, rst_dir):
    with open(yaml_path, "r", encoding="utf-8") as f:
        data = yaml.safe_load(f)
    cmds = data.get("commands", [])

    groups = defaultdict(list)
    for cmd in cmds:
        rst_text = PAGE_TMPL.render(cmd=cmd)
        fname = f"{cmd['command']}.rst"
        with open(os.path.join(rst_dir, fname), "w", encoding="utf-8") as fo:
            fo.write(rst_text)
        groups[group_key(cmd["command"])].append(cmd["command"])

    # 主索引 + 分组索引
    index_lines = ["AT Manual", "=========", "", ".. toctree::", "   :maxdepth: 1", ""]
    for g in sorted(groups.keys()):
        grp_name = f"index_{g}.rst"
        index_lines.append(f"   {grp_name[:-4]}")
        glines = [g, "=" * len(g), "", ".. toctree::", "   :maxdepth: 1", ""]
        for c in groups[g]:
            glines.append(f"   {c}")
        with open(os.path.join(rst_dir, grp_name), "w", encoding="utf-8") as fo:
            fo.write("\n".join(glines))

    with open(os.path.join(rst_dir, "index.rst"), "w", encoding="utf-8") as fo:
        fo.write("\n".join(index_lines))

    print(f"✅ RST 已生成到 {rst_dir}（含分组索引 + 强化模板）")

yaml_to_rst(YAML_OUT, RST_DIR)

✅ RST 已生成到 data\rst_output（含分组索引 + 强化模板）


## Step 4 — 清理并初始化 Sphinx（现代主题 + 自定义 CSS）
- 删除旧 `docs/`，重新 `sphinx-quickstart`；  
- 切换 `sphinx_book_theme` 主题；  
- 启用 `_static/custom.css`。

In [6]:
if os.path.exists(DOCS_DIR):
    print("⚠️ 检测到旧 docs/，正在清理...")
    shutil.rmtree(DOCS_DIR)
    print("✅ 已删除旧 docs/")

!sphinx-quickstart {DOCS_DIR} --sep --project "AT Command Manual" --author "Doc Team" --release "1.0" -q

# 主题与 CSS 设置
conf_py = os.path.join(DOCS_DIR, "source", "conf.py")
with open(conf_py, "a", encoding="utf-8") as f:
    f.write('\n')
    f.write('html_theme = "sphinx_book_theme"\n')
    f.write('html_theme_options = {\n')
    f.write('   "repository_url": "https://github.com/Bingboom/docs-as-code-learning",\n')
    f.write('   "use_repository_button": True,\n')
    f.write('}\n')
    f.write('html_static_path = ["_static"]\n')
    f.write('html_css_files = ["custom.css"]\n')
    f.write('def setup(app):\n    app.add_css_file("custom.css")\n')

static_dir = os.path.join(DOCS_DIR, "source", "_static")
os.makedirs(static_dir, exist_ok=True)
custom_css = """
/* --- Global typography --- */
body { line-height: 1.6; }
h1, h2, h3 { font-weight: 600; }

/* --- Admonitions --- */
.admonition.tip { background: #eaf7ff; border-left: 4px solid #1476ff; }
.admonition.important { background: #fff5e6; border-left: 4px solid #ff9f1a; }

/* --- Parameter tables --- */
table.cmd-param-table, .cmd-param-table { width: 100%; }
.cmd-param-table th { background-color: #f2f2f2; }
.cmd-param-table td, .cmd-param-table th { padding: 6px 10px; }

/* --- Code blocks --- */
.highlight pre { border-radius: 6px; padding: 10px; }
"""
with open(os.path.join(static_dir, "custom.css"), "w", encoding="utf-8") as f:
    f.write(custom_css)

shutil.copytree(RST_DIR, os.path.join(DOCS_DIR, "source"), dirs_exist_ok=True)
print("✅ Sphinx 初始化完成并复制 RST + 注入主题与 CSS")
print_tree("docs")


Finished: An initial directory structure has been created.

You should now populate your master file c:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\pipeline-1009\docs\source\index.rst and create other documentation
source files. Use the Makefile to build the docs, like so:
   make builder
where "builder" is one of the supported builders, e.g. html, latex or linkcheck.

✅ Sphinx 初始化完成并复制 RST + 注入主题与 CSS
docs/
  make.bat
  Makefile
  build/
  source/
    AT+CCID.rst
    AT+CCLK.rst
    AT+CEREG.rst
    AT+CESQ.rst
    AT+CFUN.rst
    AT+CGATT.rst
    AT+CGDCONT.rst
    AT+CGMM.rst
    AT+CGSN.rst
    AT+CIMI.rst
    AT+CLCK.rst
    AT+CMGD.rst
    AT+CMGF.rst
    AT+CMGL.rst
    AT+CMGR.rst
    AT+CMGS.rst
    AT+CMGW.rst
    AT+CMSS.rst
    AT+CMUX.rst
    AT+CNMI.rst
    AT+COPS.rst
    AT+CPIN.rst
    AT+CPMS.rst
    AT+CPWD.rst
    AT+CREG.rst
    AT+CSCA.rst
    AT+CSCS.rst
    AT+CSDH.rst
    AT+CSMP.rst
    AT+CSMS.rst
    AT+CSQ.rst
    AT+GMM.rst
    AT

## Step 5 — 构建 HTML（失败自动回退 docutils 版本）

In [7]:
SRC_DIR = os.path.join(DOCS_DIR, "source")
BUILD_DIR = os.path.join(DOCS_DIR, "build", "html")
os.makedirs(BUILD_DIR, exist_ok=True)

def build_html_with_fallback():
    print("📦 开始构建 HTML ...")
    cmd = [sys.executable, "-m", "sphinx", "-b", "html", SRC_DIR, BUILD_DIR]
    p = subprocess.run(cmd, capture_output=True, text=True)
    print(p.stdout); print(p.stderr)
    if p.returncode == 0 and os.path.exists(os.path.join(BUILD_DIR, "index.html")):
        print("✅ HTML 构建成功 → docs/build/html/index.html")
        return True

    print("❌ 初次构建失败，尝试回退 docutils 并重试 ...")
    _ = subprocess.run([sys.executable, "-m", "pip", "install", "--quiet", "docutils<0.21"])
    p2 = subprocess.run(cmd, capture_output=True, text=True)
    print(p2.stdout); print(p2.stderr)
    if p2.returncode == 0 and os.path.exists(os.path.join(BUILD_DIR, "index.html")):
        print("✅ 回退后构建成功 → docs/build/html/index.html")
        return True

    print("❌ 构建失败，请检查上面的日志输出。")
    return False

build_html_with_fallback()

📦 开始构建 HTML ...


Exception in thread Thread-8 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\txiab\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "c:\Users\txiab\Documents\Git-folder\Building-docs\docs-as-code-learning\.venv\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\txiab\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\txiab\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1615, in _readerthread
    buffer.append(fh.read())
                  ~~~~~~~^^
UnicodeDecodeError: 'gbk' codec can't decode byte 0x8d in position 404: illegal multibyte sequence


Running Sphinx v8.2.3
loading translations [en]... done
building [mo]: targets for 0 po files that are out of date
writing output... 
building [html]: targets for 64 source files that are out of date
updating environment: [new config] 64 added, 0 changed, 0 removed
reading sources... [  2%] AT+CCID
reading sources... [  3%] AT+CCLK
reading sources... [  5%] AT+CEREG
reading sources... [  6%] AT+CESQ
reading sources... [  8%] AT+CFUN
reading sources... [  9%] AT+CGATT
reading sources... [ 11%] AT+CGDCONT
reading sources... [ 12%] AT+CGMM
reading sources... [ 14%] AT+CGSN
reading sources... [ 16%] AT+CIMI
reading sources... [ 17%] AT+CLCK
reading sources... [ 19%] AT+CMGD
reading sources... [ 20%] AT+CMGF
reading sources... [ 22%] AT+CMGL
reading sources... [ 23%] AT+CMGR
reading sources... [ 25%] AT+CMGS
reading sources... [ 27%] AT+CMGW
reading sources... [ 28%] AT+CMSS
reading sources... [ 30%] AT+CMUX
reading sources... [ 31%] AT+CNMI
reading sources... [ 33%] AT+COPS
reading sources

True

## 🟢 Step 6 — 一键执行 `run_all(clean=True)`

In [8]:
def run_all(clean=True):
    _ = extract_word_to_csv(IN_WORD, CSV_OUT)
    csv_to_yaml(CSV_OUT, YAML_OUT)
    yaml_to_rst(YAML_OUT, RST_DIR)
    if clean and os.path.exists(DOCS_DIR):
        print("⚠️ run_all: 清理旧 docs/ ...")
        shutil.rmtree(DOCS_DIR)
    get_ipython().system('sphinx-quickstart docs --sep --project "AT Command Manual" --author "Doc Team" --release "1.0" -q')
    conf_py = os.path.join(DOCS_DIR, "source", "conf.py")
    with open(conf_py, "a", encoding="utf-8") as f:
        f.write('\nhtml_theme = "sphinx_book_theme"\n')
        f.write('html_theme_options = {\n   "repository_url": "https://github.com/Bingboom/docs-as-code-learning",\n   "use_repository_button": True,\n}\n')
        f.write('html_static_path = ["_static"]\n')
        f.write('html_css_files = ["custom.css"]\n')
        f.write('def setup(app):\n    app.add_css_file("custom.css")\n')
    static_dir = os.path.join(DOCS_DIR, "source", "_static")
    os.makedirs(static_dir, exist_ok=True)
    with open(os.path.join(static_dir, "custom.css"), "w", encoding="utf-8") as f:
        f.write("/* same as Step 4 css */")
    shutil.copytree(RST_DIR, os.path.join(DOCS_DIR, "source"), dirs_exist_ok=True)
    build_html_with_fallback()
    print("\n✅ 全流程完成。HTML 查看：docs/build/html/index.html")
    print("📝 解析日志：parse_log.txt")

print("准备就绪。按顺序运行各 Step，或直接 run_all(clean=True)。")

准备就绪。按顺序运行各 Step，或直接 run_all(clean=True)。
